# High-level LSTM TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params_lstm import *
from common.utils import *

In [2]:
# export TF_ENABLE_WINOGRAD_NONFUSED=1
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = "1"
print(os.environ['TF_ENABLE_WINOGRAD_NONFUSED'])

1


In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
Tensorflow:  1.3.0
GPU:  ['Tesla K80', 'Tesla K80']


In [4]:
def create_symbol():
    word_vectors = tf.contrib.layers.embed_sequence(
          X, vocab_size=MAXFEATURES, embed_dim=EMBEDSIZE)
    word_list = tf.unstack(word_vectors, axis=1)
    cell = tf.contrib.rnn.GRUCell(NUMHIDDEN)
    _, encoding = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)
    logits = tf.layers.dense(encoding, 2, activation=None, name='output')
    return logits

In [5]:
def init_model(m):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(LR, BETA_1, BETA_2, EPS)
    training_op = optimizer.minimize(loss)
    return training_op

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)# CNTK format
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 25000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 int32 int32
CPU times: user 6.02 s, sys: 320 ms, total: 6.34 s
Wall time: 7.47 s


In [7]:
%%time
# Place-holders
X = tf.placeholder(tf.int32, shape=[None, MAXLEN])
y = tf.placeholder(tf.int32, shape=[None])
# Initialise model
sym = create_symbol()

CPU times: user 10.1 s, sys: 112 ms, total: 10.3 s
Wall time: 10.3 s


In [8]:
%%time
model = init_model(sym)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 10.9 s, sys: 888 ms, total: 11.8 s
Wall time: 11.8 s


In [9]:
%%time
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.9375
1 Train accuracy: 0.953125
2 Train accuracy: 0.96875
CPU times: user 1min 16s, sys: 2.14 s, total: 1min 18s
Wall time: 1min 19s


In [10]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 2min 46s, sys: 2.75 s, total: 2min 49s
Wall time: 2min 47s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.860536858974
